In [159]:
import pandas as pd
import re        #正規運算式
import math
import time
import numpy as np 
import csv
from datetime import datetime, timedelta
from datetime import date
import pickle
import os

In [160]:
bbs = pd.read_excel('./bbs.xlsx')
forum = pd.read_excel('./forum.xlsx')
news = pd.read_excel('./news.xlsx')
collections = pd.concat([bbs,forum,news],axis=0, ignore_index=True,sort=False)

In [161]:
Indexprice = pd.read_excel('./Index_3.xlsx')

In [162]:
def normal(xdict):
    temp = []
    for key in xdict.keys():
        temp.append(xdict[key])
    n = np.asarray(temp)
    if np.sum(n * n) == 0:
        outvec = temp
    else:
        outvec = n / (np.sum(n * n) ** 0.5)
        outvec = outvec.tolist()
    return outvec

In [163]:
def remove_stopwords(content):
    content = re.sub(r'[^\w]','',content)        #移除非文字字元(符號)
    content = re.sub(r'[A-Za-z0-9]','',content)  #移除英文&數字
    '''
    for s in stopword:
        print(s)
        re.sub(s,'',content)
    '''
    return content

In [189]:
def getready(wn):
    global xs,start_date,end_date,model,keywb
    #incwordbank = np.load('./bt/increaseword_period'+str(ver)+'_3d1.2atr3_wb.pkl',allow_pickle = True)
    #decwordbank = np.load('./bt/decreaseword_period'+str(ver)+'_3d1.2atr3_wb.pkl',allow_pickle = True)
    xs = np.load('./bt/btxa/output_X_period'+str(wn)+'_3d1.2atr3_dc21.npy',allow_pickle = True)
    #wordbank = dict(incwordbank,**decwordbank)
    sy = 2016
    sm = 4 + wn
    ey = 2016
    em = 5 + wn
    while sm > 12:
        sy = sy + 1
        sm = sm - 12      
    while em > 12:
        ey = ey + 1
        em = em - 12
        
    start_date = datetime(sy,sm,1)
    end_date = datetime(ey,em,1)
    if wn == 32:
        end_date = datetime(2018,12,26)
        
    keywb = xs[0].keys()

In [190]:
for ver in range(32,33):
    getready(ver)
    
    def arttodict(art):
        temp = {}
        for word in keywb:
            temp[word] = art.count(word)
        return temp
    
    with open('./bt/model/output_X_period'+str(ver)+'_3d1.2model.pickle','rb') as f:
        model = pickle.load(f)
        
    filtered_list_apple_noENG = []
    for index,value in collections.iterrows():
        if (start_date <= value["post_time"].to_pydatetime()) and (end_date >=value["post_time"].to_pydatetime()):
            if ("蘋果" in (str(value["content"])+str(value["title"]))) or("蘋概股" in (str(value["content"])+str(value["title"]))):
                filtered_list_apple_noENG.append([value["post_time"].to_pydatetime(),str(value["title"])+str(value["content"])])

    Index_date = []
    for i in range(Indexprice.shape[0]):
        Index_date.append(Indexprice.iloc[i][0].date())

    for i in range(len(filtered_list_apple_noENG)):
        if filtered_list_apple_noENG[i][0].date() >= datetime(2016,1,4).date():
            while filtered_list_apple_noENG[i][0].date() not in Index_date:
                filtered_list_apple_noENG[i][0] = filtered_list_apple_noENG[i][0] - timedelta(days=1)
    
    Index_date_shred = []
    shred_dict = {}
    for d in Index_date:
        if d >= start_date.date() and d < end_date.date():
            Index_date_shred.append(d)
            shred_dict[d] = []
        elif d >= end_date.date():
            break
    print(Index_date_shred)
    
    print('文章數:',len(filtered_list_apple_noENG))
    for arts in filtered_list_apple_noENG:
        if arts[0].date() in Index_date_shred:
            shred_dict[arts[0].date()].append(remove_stopwords(arts[1]))
    
    vector_dict = {}
    for d in Index_date_shred:
        vector_dict[d] = []
    #print(shred_dict)

    for key in vector_dict:
        for art in shred_dict[key]:
            vector_dict[key].append(arttodict(art))
    
    vote = []
    for daykey in vector_dict:
        index = Index_date.index(daykey)
        res = np.array([0,0])
        for vec in vector_dict[daykey]:
            #print(vec)
            nvec = [normal(vec)]
            ticket = model.predict(nvec)
            res = res +ticket
        temp = [index,res]
        vote.append(temp)
    
    print(vote)
    board = 0
    for indv in vote:
        artnum = np.sum(indv[1])
        board += artnum
    print('判別文章數:',board)
    
    finalvote = []
    for v in vote:
        index = v[0]
        pos_tic = (v[1][0,0])
        neg_tic = (v[1][0,1])
        total_tic = pos_tic + neg_tic
        if total_tic!=0:
            if (pos_tic/total_tic) > 0.6 :
                finalvote.append([index,1])
            elif (pos_tic/total_tic) < 0.4 :
                finalvote.append([index,-1])
            else:
                finalvote.append([index,0])
        else:
            finalvote.append([index,0])
    
    TP = 0
    FP = 0
    TN = 0
    FN = 0
    giveup = 0
    for fv in finalvote:
        date = fv[0]
        if fv[1] == 0:
            giveup = giveup + 1
        elif fv[1] == 1:
            if Indexprice.iloc[date+3,4] >= Indexprice.iloc[date,4]:
                TP = TP + 1
            elif Indexprice.iloc[date+3,4] < Indexprice.iloc[date,4]:
                FP = FP + 1
        elif fv[1] == -1:
            if Indexprice.iloc[date+3,4] >= Indexprice.iloc[date,4]:
                FN = FN + 1
            elif Indexprice.iloc[date+3,4] < Indexprice.iloc[date,4]:
                TN = TN + 1

    output = [TP,FP,TN,FN,giveup]
    rowname = ['TP','FP','TN','FN','giveup']
    dataframe = pd.DataFrame({'ROWname':rowname,'Value':output})
    print(dataframe)
    dataframe.to_csv('hardtrainresult'+str(ver)+'.csv',index=False,sep=',')

[datetime.date(2018, 12, 3), datetime.date(2018, 12, 4), datetime.date(2018, 12, 5), datetime.date(2018, 12, 6), datetime.date(2018, 12, 7), datetime.date(2018, 12, 10), datetime.date(2018, 12, 11), datetime.date(2018, 12, 12), datetime.date(2018, 12, 13), datetime.date(2018, 12, 14), datetime.date(2018, 12, 17), datetime.date(2018, 12, 18), datetime.date(2018, 12, 19), datetime.date(2018, 12, 20), datetime.date(2018, 12, 21), datetime.date(2018, 12, 22), datetime.date(2018, 12, 24), datetime.date(2018, 12, 25)]
文章數: 973
[[716, array([[21, 46]])], [717, array([[16, 51]])], [718, array([[25, 49]])], [719, array([[22, 41]])], [720, array([[15, 34]])], [721, array([[24, 39]])], [722, array([[30, 59]])], [723, array([[14, 25]])], [724, array([[11, 26]])], [725, array([[23, 57]])], [726, array([[14, 34]])], [727, array([[10, 35]])], [728, array([[11, 23]])], [729, array([[ 8, 21]])], [730, array([[13, 24]])], [731, array([[17, 33]])], [732, array([[ 4, 18]])], [733, array([[15, 30]])]]
判別文章

In [150]:
filtered_list_apple_noENG = []
for index,value in collections.iterrows():
    if (start_date <= value["post_time"].to_pydatetime()) and (end_date >=value["post_time"].to_pydatetime()):
        if ("蘋果" in (str(value["content"])+str(value["title"]))) or("蘋概股" in (str(value["content"])+str(value["title"]))):
            filtered_list_apple_noENG.append([value["post_time"].to_pydatetime(),str(value["title"])+str(value["content"])])

Index_date = []
for i in range(Indexprice.shape[0]):
    Index_date.append(Indexprice.iloc[i][0].date())

for i in range(len(filtered_list_apple_noENG)):
    if filtered_list_apple_noENG[i][0].date() >= datetime(2016,1,4).date():
        while filtered_list_apple_noENG[i][0].date() not in Index_date:
            filtered_list_apple_noENG[i][0] = filtered_list_apple_noENG[i][0] - timedelta(days=1)

In [151]:
Index_date_shred = []
shred_dict = {}
for d in Index_date:
    if d >= start_date.date() and d < end_date.date():
        Index_date_shred.append(d)
        shred_dict[d] = []
    elif d >= end_date.date():
        break
print(Index_date_shred)

[datetime.date(2016, 4, 1), datetime.date(2016, 4, 6), datetime.date(2016, 4, 7), datetime.date(2016, 4, 8), datetime.date(2016, 4, 11), datetime.date(2016, 4, 12), datetime.date(2016, 4, 13), datetime.date(2016, 4, 14), datetime.date(2016, 4, 15), datetime.date(2016, 4, 18), datetime.date(2016, 4, 19), datetime.date(2016, 4, 20), datetime.date(2016, 4, 21), datetime.date(2016, 4, 22), datetime.date(2016, 4, 25), datetime.date(2016, 4, 26), datetime.date(2016, 4, 27), datetime.date(2016, 4, 28), datetime.date(2016, 4, 29)]


In [152]:
print(len(filtered_list_apple_noENG))
for arts in filtered_list_apple_noENG:
    if arts[0].date() in Index_date_shred:
        shred_dict[arts[0].date()].append(remove_stopwords(arts[1]))

405


In [153]:
vector_dict = {}
for d in Index_date_shred:
    vector_dict[d] = []
#print(shred_dict)

for key in vector_dict:
    for art in shred_dict[key]:
        vector_dict[key].append(arttodict(art))

In [154]:
vote = []
for daykey in vector_dict:
    index = Index_date.index(daykey)
    res = np.array([0,0])
    for vec in vector_dict[daykey]:
        #print(vec)
        nvec = [normal(vec)]
        ticket = model.predict(nvec)
        res = res +ticket
    temp = [index,res]
    vote.append(temp)

In [155]:
print(vote)
board = 0
for indv in vote:
    artnum = np.sum(indv[1])
    board += artnum
print(board)

[[57, array([[21,  2]])], [58, array([[18,  0]])], [59, array([[11,  2]])], [60, array([[18,  1]])], [61, array([[16,  0]])], [62, array([[4, 0]])], [63, array([[14,  1]])], [64, array([[12,  0]])], [65, array([[26,  1]])], [66, array([[15,  4]])], [67, array([[17,  6]])], [68, array([[21,  1]])], [69, array([[13,  2]])], [70, array([[20,  1]])], [71, array([[28,  2]])], [72, array([[21,  2]])], [73, array([[39,  2]])], [74, array([[20,  1]])], [75, array([[35,  3]])]]
400


In [156]:
finalvote = []
for v in vote:
    index = v[0]
    pos_tic = (v[1][0,0])
    neg_tic = (v[1][0,1])
    total_tic = pos_tic + neg_tic
    if total_tic!=0:
        if (pos_tic/total_tic) > 0.6 :
            finalvote.append([index,1])
        elif (pos_tic/total_tic) < 0.4 :
            finalvote.append([index,-1])
        else:
            finalvote.append([index,0])
    else:
        finalvote.append([index,0])
    

In [157]:
TP = 0
FP = 0
TN = 0
FN = 0
giveup = 0
for fv in finalvote:
    date = fv[0]
    print(Indexprice.iloc[date+3,4],Indexprice.iloc[date,4])
    if fv[1] == 0:
        giveup = giveup + 1
    elif fv[1] == 1:
        if Indexprice.iloc[date+3,4] >= Indexprice.iloc[date,4]:
            TP = TP + 1
        elif Indexprice.iloc[date+3,4] < Indexprice.iloc[date,4]:
            FP = FP + 1
    elif fv[1] == -1:
        if Indexprice.iloc[date+3,4] >= Indexprice.iloc[date,4]:
            FN = FN + 1
        elif Indexprice.iloc[date+3,4] < Indexprice.iloc[date,4]:
            TN = TN + 1
            
output = [TP,FP,TN,FN,giveup]
rowname = ['TP','FP','TN','FN','giveup']
dataframe = pd.DataFrame({'ROWname':rowname,'Value':output})
print(dataframe)
dataframe.to_csv('playbtresult'+str(ver)+'.csv',index=False,sep=',')


108.96105696251746 111.54599024311085
110.42661879502758 110.29071202301624
106.60195881551502 111.2254936762782
108.15880131579394 108.96105696251746
107.21861043777534 110.42661879502758
109.80996717276726 106.60195881551502
108.42081063994509 108.15880131579394
105.78313741797416 107.21861043777534
104.70399707901863 109.80996717276726
105.06675321426277 108.42081063994509
100.86615211416168 105.78313741797416
98.84073551933629 104.70399707901863
101.65150393352063 105.06675321426277
101.14269872984643 100.86615211416168
100.68359077862412 98.84073551933629
101.16534985851497 101.65150393352063
101.56935133252871 101.14269872984643
100.91213052459341 100.68359077862412
101.32931698395153 101.16534985851497
  ROWname  Value
0      TP      8
1      FP     11
2      TN      0
3      FN      0
4  giveup      0
